# Customise NZUpy's scenarios

## Overview of customisation

This notebook demonstrates how to create and customise multiple scenarios in NZUpy. We'll explore:

1. Customising inputs with our own values not featured as an available 'config' in the model's standard imported CSVs
2. Generating comparative charts
3. Using our own custom functions to edit input parameters
4. The impact of some important model variables that can affect results

## 1. Set Up the NZUpy Model

First, let's import the necessary libraries and initialise the model.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "02_customise_scenarios"
os.makedirs(output_dir, exist_ok=True)

# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Define the scope

For this run, we'll look at an important parameter that can affect price outcomes - liquidity factor which sets a limit in any given year on how many non-surplus stockpile units can be used to make-up for short-falls in supply. 

By non-surplus units, we mean for example, units held by production foresters who need these units to cover their harvest liabilities in future. These differ from the 'surplus' units defined by the Climate Commission and Government as those that don't have clear/valid purpose for being held in private accounts.

In [ ]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Define multiple scenarios for different liquidity factors & prime the model
NZU.define_scenarios(['Low Liquidity', 'Medium Liquidity', 'High Liquidity'])
NZU.prime()

## 3. Configure Scenarios with Different Liquidity Factors

We'll set up three scenarios with different liquidity factors (0.10, 0.15, 0.20) to see how this key parameter affects model outcomes. The liquidity factor determines what proportion of the non-surplus stockpile is available for use each year.

In [ ]:
# Start with central configs for all scenarios
NZU.use_central_configs(0)  # Low Liquidity scenario
NZU.use_central_configs(1)  # Medium Liquidity scenario 
NZU.use_central_configs(2)  # High Liquidity scenario

# Customise liquidity factors
NZU.set_parameter('liquidity_factor', 0.10, component='stockpile', scenario_index=0)
NZU.set_parameter('liquidity_factor', 0.15, component='stockpile', scenario_index=1)
NZU.set_parameter('liquidity_factor', 0.20, component='stockpile', scenario_index=2)

# Verify the parameter settings
print("Liquidity factors set for each scenario:")
for i, scenario in enumerate(NZU.scenarios):
    param = NZU.show_parameter('liquidity_factor', 'stockpile', scenario_name=scenario)
    print(f"  {scenario}: {param:.2f}")

## 4. Run the Model and Generate Comparison Charts

Let's run the model with our different liquidity scenarios and visualise the results.

In [ ]:
# Run the model
results = NZU.run()

In [ ]:
# Load chart generator
from model.utils.chart_generator import ChartGenerator

# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# Generate carbon price comparison chart
price_chart = chart_gen.carbon_price_chart()
display(price_chart)
price_chart.write_image(str(output_dir / "liquidity_price_comparison.png"))

# Generate stockpile balance comparison
stockpile_chart = chart_gen.stockpile_balance_chart()
display(stockpile_chart)
stockpile_chart.write_image(str(output_dir / "liquidity_stockpile_comparison.png"))

# Generate supply-demand balance comparison
balance_chart = chart_gen.supply_demand_balance_chart()
display(balance_chart)
balance_chart.write_image(str(output_dir / "liquidity_balance_comparison.png"))

## 5. Discussion of Results

The charts above reveal several key insights about how liquidity factor affects the NZ ETS:

1. **Carbon Price Impact**: Higher liquidity factors result in lower carbon prices as more non-surplus units become available for compliance, reducing the need for emissions reductions through price signals.

2. **Stockpile Dynamics**: With lower liquidity (0.10), we see a faster depletion of the surplus stockpile, followed by a more gradual use of the non-surplus component. The higher liquidity scenarios (0.15, 0.20) show a more gradual decline in total stockpile balance.

3. **Payback Requirements**: Notice that towards the end of the model period, all scenarios show an uptick in stockpile balance. This demonstrates the model's payback mechanism in action - borrowed units from the non-surplus component must be paid back over the payback period (default is 25 years).

4. **Price Ramping**: In the low liquidity scenario, the model may ramp up prices more aggressively to address supply-demand imbalances when fewer units are available from the stockpile. This occurs because the model applies a severe penalty to supply shortfalls.

If price ramping seems excessive for your use case, this penalty behaviour can be adjusted through the optimiser settings:

```python
# Example (not run): Turn off shortfall penalty
NZU.config.optimiser.penalise_shortfalls = False
```

## 6. Modelling a Scarcity Scenario

Now, let's reset our model and create a new set of scenarios to explore a hypothetical situation with temporary market scarcity in the mid-2020s followed by oversupply.

In [ ]:
# Reinitialise the model
NZU = NZUpy(data_dir=data_dir)
NZU.define_time(2024, 2050)

# Define two scenarios: baseline and scarcity scenario
NZU.define_scenarios(['Baseline', 'Scarcity Then Surplus'])

# Prime the model
NZU.prime()

# Configure scenarios
NZU.use_central_configs(0)  # Set all components to central for baseline scenario
NZU.use_central_configs(1)  # Start with central configs for scarcity scenario

Now, let's modify the second scenario to create scarcity by reducing auction volumes in the mid-2020s and then increasing them later to create surplus.

In [ ]:
# Get the auction data for your scenario
auction_data = NZU.show_inputs('auction', scenario_name='Scarcity Then Surplus')

# Get the base volume series and create a copy to modify
base_volumes = auction_data['base_volume'].copy()

# Modify the volumes as needed
base_volumes.loc[2025] = 0.0  # Set 2025 volume to zero
base_volumes.loc[2030:2050] = base_volumes.loc[2030:2050] * 1.1  # Increase volumes by 10% from 2030-2050

# Update the auction data in the model
NZU.set_series('base_volume', base_volumes, 'auction', scenario_name='Scarcity Then Surplus')

# Verify the changes by showing the inputs again
modified_data = NZU.show_inputs('auction', scenario_name='Scarcity Then Surplus')
print("Modified auction volumes:")
print(modified_data['base_volume'].head(10))  # Show first 10 years

Now let's adjust the price control parameters to manage the price trajectory during scarcity and surplus periods.

In [ ]:
# Load the "scarcity_then_surplus" price control configuration only for the second scenario
NZU.use_price_control_config('scarcity_then_surplus', scenario_index=1)

# Run the model with our scenarios
results = NZU.run()

## 7. Visualising Scarcity Scenario Results

Let's create charts to visualise how our scarcity scenario affects prices and market dynamics.

In [ ]:
# Initialize chart generator
chart_gen = ChartGenerator(NZU)

# Create comparison page for specific chart type
from model.interface.chart_display import create_comparison_page

# Dictionary mapping scenario names to model instances
models = {
    'Baseline': NZU,  # Your baseline model
    'Scarcity Then Surplus': NZU  # Your scarcity model
}

# Create comparison for any chart type you want
create_comparison_page(
    models=models,
    chart_type='carbon_price',  # Can be: carbon_price, emissions_pathway, supply_components, stockpile_balance, supply_demand_balance, auction_volume_revenue
    output_dir=output_dir,
    filename="price_comparison.html"
)

## 8. Discussion of Scarcity Scenario Results

The scarcity scenario demonstrates several important features of the NZ ETS model:

1. **Price Spike**: Removing auction volumes in 2025 creates a supply shortage, causing a price spike as the market adjusts. The price control parameter amplifies this effect, demonstrating how policy settings can influence price trajectories.

2. **Stockpile Utilisation**: During the scarcity period, the model draws more heavily from the stockpile to make up the supply shortfall, accelerating the depletion of the surplus component.

3. **Price Dampening**: When auction volumes increase after 2030, combined with the reduced price control parameter, prices are dampened compared to the baseline scenario. This demonstrates how the model accounts for market oversupply.

4. **Market Recovery**: The supply components chart for the scarcity scenario shows how different supply sources adjust over time to compensate for the auction volume changes.

The price control feature demonstrated here was introduced as part of MfE's most recent update to the government's excel-based NZ ETS model. This feature allows modelling of how policy interventions and market expectations can influence price formation beyond simple supply-demand dynamics.

## 9. Conclusion

This notebook has demonstrated:

1. **Creating multiple custom scenarios** with different parameter values
2. **Modifying specific parameters** like the liquidity factor to values not available in predefined CSVs
3. **Manually editing input series** to model specific market conditions
4. **Using price control parameters** to influence price trajectories
5. **Generating and interpreting comparative charts** across scenarios

These techniques allow for rich explorations of different market conditions and policy settings within the NZ ETS model.